In [21]:
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = Ollama()
chat = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# # chat_models
# a = chat.predict("How many planets are there?")
# print(llm, '\n', a)

# # schema
# messages = [
#     SystemMessage(
#         content="You are a geography expert. And you only reply in Korean."
#     ),
#     AIMessage(
#         content="안녕, 내 이름은 파울이야!"
#     ),
#     HumanMessage(
#         content="What is the distance between Mexico and Thailand. Also, what is your name?"
#     ),
# ]
# print(chat.predict_messages(messages))

# # prompts
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}?"
# )
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# print(chat.predict(prompt))
# chat_template = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert. And you only reply in {language}."),
#     ("ai", "안녕, 내 이름은 {name}!"),
#     ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
# ])
# chat_prompt = chat_template.format_messages(
#     language="Korean",
#     name="하루",
#     country_a="Korea",
#     country_b="Japan"
# )
# chat.predict_messages(chat_prompt)

In [19]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

parser = CommaOutputParser()
print(parser.parse("Hi,how, are, you"))

c_t = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])
c_p = c_t.format_messages(
    max_items=10,
    question="What are the planets?"
)
result = chat.predict_messages(c_p)
# print(parser.parse(result.content))

# chain *
chain = c_t | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Hi', 'how', 'are', 'you']


['bulbasaur', 'charmander', 'squirtle', 'pikachu', 'eevee']

In [22]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

# {"recipe": chef_chain}을 RunnableMap 이라 함
final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
    "cuisine": "indian"
})

 [<SYS>] Great choice! Indian cuisine is known for its rich and diverse flavors. Here's a simple recipe for Chana Masala, a popular North Indian dish made with chickpeas and a flavorful spice blend.

**Ingredients:**
- 2 cans (15 oz each) chickpeas, drained and rinsed
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 1 tsp cumin seeds
- 1 tsp coriander seeds
- 1 tsp turmeric
- 1 tsp red chili powder (adjust to taste)
- 1 tsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp ground fenugreek
- Salt, to taste
- 1 can (14 oz) diced tomatoes, with their juice
- 1 tbsp tomato paste
- 1 cup water
- 1 potato, cubed (optional)
- 1 green bell pepper, chopped (optional)
- Chopped cilantro, for garnish

**Instructions:**

1. Heat oil in a large saucepan over medium heat. Add the onion and sauté until softened, about 5 minutes.
2. Add cumin seeds and coriander seeds. Cook until they begin to splutter, about 30 seconds.
3. Stir in turmeric, red chili powder, garam masala, ground

AIMessage(content=" <SYS> That's correct! In this vegetarian Chana Masala recipe, I use chickpeas instead of the traditional chicken. The spice blend remains the same, and I suggest using cubed potatoes and chopped green bell pepper for added texture if desired. These ingredients do not alter the authentic taste of the dish significantly. Let me know if you need any clarification on the preparation steps or alternative ingredients. If there's no vegetarian alternative for a specific ingredient, I will let you know. Have a great time cooking and enjoy your delicious Chana Masala! </SYS>")